## How to use the Data Lab *Query Manager* Service

*Revised: May 7, 2017*

This notebook documents how to query the Data Lab via the query manager service. This can be done either from a Python script or from the command line using the <i>datalab</i> command.

### The query manager service interface

The query manager service simplifies access to the Data Lab databases. This section describes the query manager service interface in case we want to write our own code against that rather than using one of the provided tools.
The query manager service accepts an HTTP GET call to the <i>query</i> endpoint with the following parameters:

| Name | Function | Optional | Supported values |
|------|----------|----------|------------------|
| adql | The query string to run against the db | No | - |
| out | The location to save any results | Yes | 'vos://...', 'mydb://...' |
| ofmt | The output format of any results | Yes | ascii, csv, votable |
| async | Run the query asynchronously | Yes | true/false |
| profile | The server profile to use | Yes | default |

For example: /query?adql=<query>&ofmt=csv

#### Saving results
If no save location is specified (no <i>out</i> param) then the results are returned directly. A save location beginning with the 'vos://' identifier indicates a location in the user's virtual storage to save the result. A save location beginning with the 'mydb://' identifier indicates the results are to be saved to a table in the user's remote database (MyDB). 

#### Output formats
The results can be returned as whitespace delimited (<i>ascii</i>), CSV (<i>csv</i>), or in VOTable format (<i>votable</i>). Note that if the results are saved to the user's database then the output format is ignored.

#### Asynchronous queries
Long queries should be run asynchronously and the service may refuse any synchronous query attempted if the projected query time is too long. A query can be submitted asynchronously by setting the <i>async</i> parameter to <i>true</i>. A job id will then be returned.

The status of an asynchronous query can be checked by submitting an HTTP GET call to the query manager service <i>status</i> endpoint with the relevant job id as an argument: /status?jobid=<jobid>. A return value of 'COMPLETED' indicates the query has terminated. A return value of 'ERROR' indicates that there was a problem with the query.

The results of an asynchronous query (assuming that they were not saved to either the user's virtual storage or remote database) can be retrieved once the query has completed with an HTTP GET call to the query manager service <i>results</i> endpoint with the relevant job id as argument: /results?jobid=<jobid>

#### Authentication
The query manager service requires a Data Lab security token. This needs to be passed as the value of the header keyword "X-DL-AuthToken" in any HTTP GET call to the service. 

#### Profiles
There are several different backend machines serving data, etc., and it is possible that a user might want to specify a particular set to run their query on and save results to. A <i>profile</i> defines such a set and a particular profile can be specified in a query call.

### From Python code

The query manager service can be called from Python code using the <i>datalab</i> module. This provides methods to access the various query manager functions in the <i>queryMgr</i> subpackage. 

#### Initialization
This is the setup that is required to use the query manager. The first thing to do is import the relevant Python modules and also retrieve our DataLab security token (remember that this has to be included in any call to the query manager service).

In [1]:
# Standard notebook imports
from __future__ import print_function
import getpass
from dl import authClient, queryClient, storeClient

In [2]:
# Get the security token for the datalab user
token = authClient.login ('fitz',getpass.getpass('Account password: '))
if not authClient.isValidToken (token):
    print ('Error: invalid user login (%s)' % token)
else:
    print ("Login token:   %s" % token)

Account password: ········
Login token:   fitz.1007.1007.$1$o9HhivQp$KTNpE/qBGPqyuOYo2TlEM1


#### The *queryClient* class

All queries are executed through the <i>query()</i> method of the <i>queryClient</i> class. This takes as arguments:

| Argument | Description | Default  value | Allowed Values |
|----------|-------------|----------------|----------------|
| token | The login identity token | None | any valid token |
| adql | The query to be submitted to the TAP service | None | |
| sql | The query to be submitted to the DB directly | None | |
| fmt | The requested format (if any) | ascii | ascii,csv,votable,fits |
| out | The saved location (if any) | None | local filename, *vos://filename*, *mydb://tablename* |
| async | Indicates if the query is asynchronous | False | |
| profile | Indicates which service profile to query | default | |

All arguments are optional except for the *token* parameter, and one of *adql* or *sql* must be supplied.  The distinction between these two parameters is in how the *QueryManager* executes the query:  If *adql* is provided the query is sent to the TAP (Table Access Protocol) service, if *sql* is provided the query is sent directly to the database.  The choice of execution depends on whether the query string contains ADQL-specific functions, or SQL constructs or DB extensions not understood by the TAP service.  For large queries there can also be a performance difference depending on the where/how the results are saved. Note that if a *profile* is not specified then the default profile is assumed.

#### A quick query

Let's say we want to return the $gri$ magnitudes of the top 10 objects in the SMASH DR1 dataset and get it back as a CSV file:

In [9]:
query = 'select * from usno.a2 limit 10'
response = queryClient.query(token, adql = query, fmt = 'csv')
print (response)

raj2000_,dej2000_,actflag,mflag,bmag,rmag,epoch,raj2000,dej2000,id
20:59:59.592,+35:16:03.27, , ,16.700001,15.1,1951.517,314.99829999999997,35.267575000000001,1200-16738677
20:59:58.753,+35:16:04.59, , ,17.200001,16.299999,1951.517,314.99480299999999,35.267941999999998,1200-16737889
21:00:01.230,+35:16:14.01, , ,19.700001,18.799999,1951.517,315.00512300000003,35.270558999999999,1200-16740156
20:59:58.012,+35:16:08.50, , ,18.5,17,1951.517,314.991714,35.269027999999999,1200-16737180
20:59:57.298,+35:16:13.51, , ,17,14.8,1951.517,314.98873900000001,35.270420000000001,1200-16736554
20:59:58.188,+35:16:18.73, , ,17.200001,15.7,1951.517,314.99245000000002,35.27187,1200-16737350
21:00:00.732,+35:16:24.67, , ,19.700001,18.700001,1951.517,315.00304799999998,35.273519999999998,1200-16739707
21:00:03.421,+35:16:19.47, , ,15.5,13.7,1951.517,315.014253,35.272075000000001,1200-16742178
21:00:04.207,+35:16:28.53, , ,18.5,17.5,1951.517,315.01752800000003,35.274591999999998,1200-16742894
21:00:05.195,+

#### Saving results to virtual storage

Now we want to save the results from the same query to our virtual storage space instead.  By putting the query in a try-block we are able to trap errors when executing the query.  Note that running this cell multiple times will trigger an error and we use the Storage Manager client to remove the file once we are done.

In [11]:
try:
    response = queryClient.query (token, adql=query, fmt='csv', 
                                  out='vos://zmags.csv')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print (e.message)
else:
    if response is not None: 
        print (response)           # print the response
    else:
        print ("OK")

# Remove the file we just created, but list it first to show it exists
storeClient.ls (token, name='vos://zmags.csv')
storeClient.rm (token, name='vos://zmags.csv')

etext: Error: edu.caltech.vao.vospace.VOSpaceException: edu.caltech.vao.vospace.VOSpaceException: edu.caltech.vao.vospace.VOSpaceException: Unable to set the last modification time for /net/dl1/vospace/users/fitz/zmags.csv
econtent: Error: edu.caltech.vao.vospace.VOSpaceException: edu.caltech.vao.vospace.VOSpaceException: edu.caltech.vao.vospace.VOSpaceException: Unable to set the last modification time for /net/dl1/vospace/users/fitz/zmags.csv
Error in query: Error: edu.caltech.vao.vospace.VOSpaceException: edu.caltech.vao.vospace.VOSpaceException: edu.caltech.vao.vospace.VOSpaceException: Unable to set the last modification time for /net/dl1/vospace/users/fitz/zmags.csv


<Response [204]>

#### Saving results to remote database

Alternatively we may want to store the results in a table called <i>mags</i> in our remote database.

In [29]:
query = "select * from usno.b1 limit 1000"
try:
    response = queryClient.query (token, adql=query, fmt='csv', 
                                  out='mydb://mags3')
    #queryClient.list (token, table='mydb://mags3')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print (e.message)
else:
    if response is not None: 
        print (response)           # print the response
    else:
        print ("OK")

http://dlsvcs.datalab.noao.edu/query/list?table=mydb://mags3
OK


#### An asynchronous query

We now want to run a longer query, say, counting the total number of objects is USNO-B1, and need to do it asynchronously. The first thing we do is submit the query as normal but with the <i>async</i> argument indicated - this will return the id of the asynchronous job. All the previous arguments can also be used to specify where and in what format we want the query results.

In [2]:
query = 'select count(*) from usno.a2'
jobId = queryClient.query(token, adql = query, async = True)
print (jobId)

qmqmocoq5hna5nwd


We can check on the status of the job at any time:

In [34]:
status = queryClient.status(token, jobId = jobId)
print (status)

COMPLETED


If the status value is "QUEUED" then the job is waiting to be executed. If
it is "ERROR" then there was a problem with the execution. When the status value is "COMPLETED", we can get our results (assuming we did not save them to our virtual storage or remote database).

In [35]:
results = queryClient.results(token, jobId = jobId)
print (results)

COUNT
1045175762



#### Using profiles

Different datasets (or versions of the same dataset) may reside on different backend servers and a user may want to work explicitly with a (typically older) dataset. In some cases these servers will be used only by developers or those with restricted access.

The first thing to do is see what profiles are available.

In [3]:
profilelist = queryClient.list_profiles(token)
print (profilelist)

default,IRSA,HEASARC,Vizier,GAVO,SIMBAD,zeus1,SDSS-DR9,STScI-RegTAP,GALEX-DR6,dldb1


The thing to note in the output here are names such as '*GAVO*', '*SDSS-DR9*', etc;  These profiles refer to external TAP services that can be accessed using the Query Manager interface.  Only a few are configured at the moment as we work on ways to automatically discover the >100 such services and provide useful listings of what they contain, but let's see how we can query one and save the result to our Virtual Storage:

In [13]:
queryClient.set_profile('GAVO')
query = 'select top 10 * from sdssdr7.sources'
response = queryClient.query(token, adql = query, fmt = 'csv', out="vos://gavo_test.csv")
print (response)

OK


In this case we queried an SDSS DR7 table at the TAP service run by GAVO in Heidelberg.  

We can get the details of a particular profile by including the name of the profile as an argument in the <i>list_profiles</i> method:

In [3]:
queryClient.list_profiles(token, "default")

{u'accessURL': u'http://gp01.datalab.noao.edu:8080/ivoa-dal/tap',
 u'database': u'tapdb',
 u'dbport': 5432,
 u'description': u'The public NOAO Data Lab TAP Service / Database',
 u'host': u'gp01.datalab.noao.edu',
 u'mydb_database': u'mydb',
 u'password': u'datalab',
 u'type': u'datalab',
 u'user': u'dlquery',
 u'vosEndpoint': u'http://dldb1.sdm.noao.edu:8080/vospace-2.0/vospace',
 u'vosRoot': u'vos://datalab.noao!vospace',
 u'vosRootDir': u'/data/vospace/users'}

So let's try a query against the default profile first - let's get a list of all tables in the default database.  Note that in this case we are accessing the *information_schema* table of the database, this table is not included in the TAP service and so we <b>must</b> use the *sql* argument to talk directly to the database.

In [44]:
sql = 'select table_catalog, table_schema, table_name from information_schema.tables'
try:
    default = queryClient.query(token, sql=sql)
except Exception, e:
    print (e.message)
else:
    print (default)

table_catalog,table_schema,table_name
tapdb,pg_catalog,pg_statistic
tapdb,pg_catalog,pg_type
tapdb,mydb,mags_1005
tapdb,sdss_dr13,dr12qso
tapdb,public,beta_distrib
tapdb,smash_dr1,source
tapdb,public,d_distrib
tapdb,pg_catalog,pg_roles
tapdb,pg_catalog,pg_shadow
tapdb,pg_catalog,pg_authid
tapdb,pg_catalog,pg_group
tapdb,pg_catalog,pg_user
tapdb,pg_catalog,pg_rules
tapdb,pg_catalog,pg_views
tapdb,pg_catalog,pg_tables
tapdb,pg_catalog,pg_matviews
tapdb,pg_catalog,pg_indexes
tapdb,pg_catalog,pg_locks
tapdb,pg_catalog,pg_settings
tapdb,pg_catalog,pg_cursors
tapdb,pg_catalog,pg_available_extensions
tapdb,pg_catalog,pg_available_extension_versions
tapdb,pg_catalog,pg_prepared_xacts
tapdb,pg_catalog,pg_prepared_statements
tapdb,pg_catalog,pg_seclabels
tapdb,pg_catalog,pg_timezone_abbrevs
tapdb,pg_catalog,pg_timezone_names
tapdb,pg_catalog,pg_stat_all_tables
tapdb,pg_catalog,pg_stat_xact_all_tables
tapdb,pg_catalog,pg_stat_sys_tables
tapdb,pg_catalog,pg_stat_xact_sys_tables
tapdb,pg_catalog,pg

And now we'll run the same query against the '*zeus1*' profile:

In [48]:
queryClient.set_profile('zeus1')
zeus1 = queryClient.query(token, sql=sql)
print (zeus1)

table_catalog,table_schema,table_name
tapdb,pg_catalog,pg_statistic
tapdb,pg_catalog,pg_type
tapdb,mydb,mags_0
tapdb,mydb,mags_1005
tapdb,stripe82,var_lc
tapdb,pg_catalog,pg_roles
tapdb,pg_catalog,pg_shadow
tapdb,pg_catalog,pg_authid
tapdb,pg_catalog,pg_group
tapdb,pg_catalog,pg_user
tapdb,pg_catalog,pg_rules
tapdb,pg_catalog,pg_views
tapdb,pg_catalog,pg_tables
tapdb,pg_catalog,pg_matviews
tapdb,pg_catalog,pg_indexes
tapdb,pg_catalog,pg_locks
tapdb,pg_catalog,pg_settings
tapdb,pg_catalog,pg_cursors
tapdb,pg_catalog,pg_available_extensions
tapdb,pg_catalog,pg_available_extension_versions
tapdb,pg_catalog,pg_prepared_xacts
tapdb,pg_catalog,pg_prepared_statements
tapdb,pg_catalog,pg_seclabels
tapdb,pg_catalog,pg_timezone_abbrevs
tapdb,pg_catalog,pg_timezone_names
tapdb,pg_catalog,pg_stat_all_tables
tapdb,pg_catalog,pg_stat_xact_all_tables
tapdb,pg_catalog,pg_stat_sys_tables
tapdb,pg_catalog,pg_stat_xact_sys_tables
tapdb,pg_catalog,pg_stat_user_tables
tapdb,pg_catalog,pg_stat_xact_user_tab

Comparing the two outputs, we can see that there are differences in which tables are available.

## Using the *datalab* command

The <i>datalab</i> command provides an alternate command line way to work with the Query Manager through the <i>query</i> subcommand.  Similarly, other commands exist to view or access virtual storage or manage the user's login.  For a summary of available commands, type:

In [14]:
!datalab help


Usage:

    % datalab <task> [task_options]

where <task> is one of:

      addcapability - Activate a capability on a Virtual Storage container
          broadcast - broadcast a SAMP message
                 cp - copy a file in Data Lab
             dropdb - Drop a user MyDB table
               exec - launch a remote task in the Data Lab
                get - get a file from Data Lab
             launch - launch a plugin
    list_query_profiles - List the available Query Manager profiles
    list_storage_profiles - List the available Storage Manager profiles
     listcapability - List the capabilities supported by this Virtual Storage
             listdb - List the user MyDB tables
                 ln - link a file in Data Lab
              login - Login to the Data Lab
             logout - Logout of the Data Lab
                 ls - list a location in Data Lab
              mkdir - create a directory in Data Lab
              mount - mount the default Virtual Storage
            

#### Logging into the Data Lab

We need to be logged into the Data Lab to use the Query Manager if we also plan to use the Virtual Storage resources of our user account.  If you simply wish to access the data and return it for local use, you can skip this step and query for data anonymously.  Here. we'll login using a demo account name, user's may login using several identities (e.g. a shared collaboration account as well as a personal account) where only one identity is active at a time (the '*whoami*' command will print what that is).

In [2]:
!datalab login user=demo00 password=balatad
!datalab whoami

Welcome back to the Data Lab, demo00
demo00


Similarly, the '*logout*' command can be used to log out of the Data Lab.

In [10]:
!datalab logout
!datalab whoami

No user is currently logged into the Data Lab
anonymous


#### A quick data query

Again we will return the magnitudes of the top 10 objects in the SMASH DR1 object table and get it back as a CSV string (this is the default, so specification of the '*fmt*' parameter is optional)

In [26]:
!datalab query fmt='csv' \
    adql='select top 10 id,gmag,rmag,imag from smash_dr1.object'

id,gmag,rmag,imag
153.1633022,22.141075,99.989998,99.989998
153.1633023,21.493074,99.989998,99.989998
153.1632994,22.169075,99.989998,99.989998
153.1751465,99.989998,99.989998,99.989998
153.1751475,99.989998,99.989998,99.989998
153.1751438,99.989998,99.989998,99.989998
153.1632972,19.639074,99.989998,99.989998
153.1751452,99.989998,99.989998,99.989998
153.1751414,99.989998,99.989998,99.989998
153.1632943,18.752075,99.989998,99.989998



#### Saving results to virtual storage

Now we'll save the results to our virtual storage space as a file.  Note that the initial login above has saved our identity token which will be passed automatically with the query.  Afterward, we'll list the virtual storage space to confrm the file was saved.  Notice that in this query, we use the '*sql*' parameter to submit the query directly to the database since we are using the SQL 'limit 10000' syntax instead of the ADQL "top 10000' syntax.

In [8]:
# Login again to access Virtual Storage
!datalab login user=demo00 password=balatad

!datalab query out='vos://smash_mags.csv'  \
    sql='select id,gmag,rmag,imag from smash_dr1.object limit 10000'
!datalab ls name=vos://

public,smash_mags.csv,tmp


#### Saving results to remote database

And we can also save data to our remote database.  Here we also use the '*listdb*' command to show the list of tables in our MyDB, by specifying the table name we get a more descriptive listing of the table structure. 

In [3]:
!datalab query out='mydb://smash_mags34' \
    adql='select top 1000 id,gmag,rmag,imag from smash_dr1.object' 
!datalab listdb
!datalab listdb table=smash_mags

usno_b1
mags3
usno_test
smash_mags
smash_mags2
smash_mags3
test3
test31
smash_mags34

id,text,
gmag,real,
rmag,real,
imag,real,



#### An asynchronous query

Alternatively, we can run a longer asynchronous query:

In [1]:
%%script bash
export jobId=`datalab query adql='select count(*) from des_sva1.gold_catalog' async=True`
echo "Job ID = "$jobId

status=`datalab qstatus jobId=$jobId`
echo "Init status:  "$status
while [ "$status" != "COMPLETED" ]
do
    echo `date`"  "$status
    sleep 1
    status=`datalab qstatus jobId=$jobId`
done

echo "Query complete, result is:"
datalab qresults jobId=$jobId

Job ID = k0nca9l132b6ied3
Init status:  EXECUTING
Thu May 4 15:22:40 MST 2017  EXECUTING
Thu May 4 15:22:42 MST 2017  EXECUTING
Query complete, result is:
COUNT
25227559

